In [4]:
!pip install ipdb -Uqq

     |████████████████████████████████| 761 kB 34.9 MB/s 
     |████████████████████████████████| 386 kB 80.2 MB/s 
     |████████████████████████████████| 1.6 MB 62.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 8.7.0 which is incompatible.


In [5]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import ipdb

class STN3d(nn.Module):
    def __init__(self, channel):
        super(STN3d, self).__init__()
        self.debug1='debug1'
        print('debug1')
        #ipdb.set_trace(context=5)
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        print('debug2')
        #ipdb.set_trace(context=5)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64) #To jest ta normalizacja odchylenia i sredniej (?1)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024) #to jest jakiś resize...(?2)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        #ipdb.set_trace(context=5)
        
        iden = Variable(torch.from_numpy(np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32))).view(1, 9).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        ipdb.set_trace(context=5)            
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


In [6]:
stn=STN3d(channel=3)

debug1
debug2


In [10]:
data=torch.rand(5,3,1000)

In [ ]:
data.shape

torch.Size([1, 3, 10])

In [ ]:
data.dtype

torch.float32

In [ ]:
data

tensor([[[0.0617, 0.9717, 0.6529, 0.6724, 0.4914, 0.3401, 0.9187, 0.1081,
          0.3084, 0.9221],
         [0.9765, 0.2494, 0.9759, 0.7906, 0.0544, 0.7340, 0.4693, 0.9397,
          0.5571, 0.3918],
         [0.5916, 0.3439, 0.0848, 0.3073, 0.2553, 0.9789, 0.0429, 0.3736,
          0.9304, 0.7625]]])

In [ ]:
data.shape

In [ ]:
m = nn.Conv1d(1, 64, 1)
input = torch.randn(20, 16, 50)
output = m(input)
print(output.shape)

torch.Size([20, 33, 24])


In [ ]:
dir(STN3d.parameters)
#STN3d.parameters.__get__

In [ ]:
dir(STN3d)
#STN3d.__getattribute__.__call__

In [11]:
dataTrans=stn.forward(data)

> <ipython-input-5-1c062874a927>(52)forward()
     50             iden = iden.cuda()
     51         ipdb.set_trace(context=5)
---> 52         x = x + iden
     53         x = x.view(-1, 3, 3)
     54         return x

ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/bdb.py", line 345, in set_continue
    sys.settrace(None)



In [12]:
dataTrans.shape

torch.Size([5, 3, 3])